In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [2]:
mlflow.set_tracking_uri("https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow")

In [4]:
import dagshub
dagshub.init(repo_owner='maverick011', repo_name='Uber-Demand-Prediction', mlflow=True)

Initialized MLflow to track repo "maverick011/Uber-Demand-Prediction"

Repository maverick011/Uber-Demand-Prediction initialized!

In [5]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [6]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [7]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [8]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [9]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [10]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [11]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [12]:
from sklearn import set_config

set_config(transform_output="pandas")

In [13]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [14]:
encoder

,transformers,"[('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [15]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

/mnt/c/Users/jatin/Desktop/My/my skills/wsl_proj/uber/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [16]:
import optuna
import tqdm 

/mnt/c/Users/jatin/Desktop/My/my skills/wsl_proj/uber/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# set the experiment

mlflow.set_experiment("Model Selection")

2025/07/14 19:20:32 INFO mlflow.tracking.fluent: Experiment with name 'Model Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/be86187d05574ccb83f9d060e8ffbd27', creation_time=1752492031440, experiment_id='0', last_update_time=1752492031440, lifecycle_stage='active', name='Model Selection', tags={}>

In [19]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb,
                                tree_method='gpu_hist',  
                                gpu_id=0)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())

        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [20]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2025-07-14 19:35:31,085] A new study created in memory with name: model_selection


🏃 View run amusing-crab-73 at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/a5b383a25f7247729aa5d2f171d03bd6
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-07-14 19:35:46,610] Trial 0 finished with value: 0.3466868818531784 and parameters: {'model_name': 'RF', 'n_estimators_rf': 50, 'max_depth_rf': 4}. Best is trial 0 with value: 0.3466868818531784.


🏃 View run stately-skink-18 at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/0251ec1de9954107b4a0bfb9eefd3a1d
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-07-14 19:35:56,924] Trial 2 finished with value: 0.34297658634125283 and parameters: {'model_name': 'RF', 'n_estimators_rf': 10, 'max_depth_rf': 4}. Best is trial 2 with value: 0.34297658634125283.


...

🏃 View run overjoyed-doe-22 at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/8bdcfe1456f840bea7787eb08b8bdbaf
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0
🏃 View run persistent-bass-823 at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/4e911c1d705e44608acf8af2a5e01553
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-07-14 19:41:50,985] Trial 47 finished with value: 0.959055680443814 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.02183155063060685}. Best is trial 9 with value: 0.07934790285463403.
[I 2025-07-14 19:41:53,042] Trial 46 finished with value: 0.9580102484227003 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.02185872912719904}. Best is trial 9 with value: 0.07934790285463403.


🏃 View run skillful-donkey-900 at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/2d679f188656419f8c179e3ce13f973b
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0
🏃 View run sneaky-quail-580 at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/12c26711b7254d938843760ba4ba4685
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-07-14 19:42:00,342] Trial 48 finished with value: 0.7227710675529212 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.025529484169621287}. Best is trial 9 with value: 0.07934790285463403.
[I 2025-07-14 19:42:01,386] Trial 49 finished with value: 0.8631026006701731 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.023167224141303323}. Best is trial 9 with value: 0.07934790285463403.


🏃 View run best_model at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/1600fc0640f64031a6f2544a3dfa9201
🧪 View experiment at: https://dagshub.com/maverick011/Uber-Demand-Prediction.mlflow/#/experiments/0


In [21]:
# best value

study.best_value

0.07934790285463403

In [22]:
# best parameters

study.best_params


{'model_name': 'LR'}

In [23]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
LR      24
RF      11
GBR      8
XGBR     7
Name: count, dtype: int64

In [29]:
from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [30]:
plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [31]:
plot_parallel_coordinate(study, params=["model_name"])

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [ ]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

In [ ]:
lr.coef_

In [ ]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss
        

In [ ]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

In [ ]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

In [ ]:
# best parameters

study.best_params

In [ ]:
# best value

study.best_value